# Solution 1
In this solution I used the following data structures and functions:
- <strong>solution</strong> is a numpy matrix where each row represents a knapsack. There actually is one more row than the number of kanpsacks and it represents the left-out items. So the row 0 contains the items which have not been taken. At first each item is left out, I also tried to start with a randomized solution but it could just not converge.
- <strong>tweak</strong> is a funtion that tries to tweak the solution by making a random number of swaps of random items each time. So when tweak is called a random number of random items is selected and they are randomly moved between the knapsacks (even between knapsack 0, so the left out items).
- <strong>fitness</strong> is a function that computes the fitness of a solution. Invalid solutions have negative fitness.

In [12]:
import numpy as np
from random import randint, sample
from copy import deepcopy
from tqdm import tqdm

In [13]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 3
NUM_DIMENSIONS = 2

In [14]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))
print("Values:\n", VALUES)
print("Weights:\n", WEIGHTS)
print("Constraints:\n", CONSTRAINTS)

Values:
 [59 99 39]
Weights:
 [[18 28]
 [ 4 85]
 [57 32]]
Constraints:
 [[ 64 142]
 [100  74]]


In [15]:
def tweak(solution: np.array) -> np.array:
  s1 = deepcopy(solution)
  # after deepcopying the solution we can perform a random number of swaps
  # of the items in the knapsack

  # we start by picking a random number of items to swap
  random_items = sample(range(0, NUM_ITEMS), k=randint(1, NUM_ITEMS))

  for item in random_items:
    # consider 2 different random bags (if b=0 then we are choosing a left out item)
    b1 = np.where(solution[:, item])[0][0]
    while True:
      b2 = randint(0, NUM_KNAPSACKS-1)
      if b2 != b1:
        break
    # swap the item in the two bags
    s1[b1][item], s1[b2][item] = solution[b2][item], solution[b1][item]

  return s1


In [ ]:
def fitness(solution: np.array) -> int:
  # at first we need to compute if the solution is feasible and if it is not we return a negative fitness
  if np.any(solution.sum(axis=0) > 1):
    return -2   # if an object apperas in multiple knapsacks than the algorithm itself is flawed
    
  for k in range(1, NUM_KNAPSACKS+1):        # for each knapsack
    
    items_in_knapsack = solution[k]

    if np.any(items_in_knapsack):           # if there are some items in the knapsack we can compute the total weight
        
        total_weight = WEIGHTS[items_in_knapsack].sum(axis=0)   # compute the total weight for each dimension
        
        if np.any(total_weight > CONSTRAINTS[k-1]):   
            return -1   # the solution violates the weight constraint


  # If the constraints are not violated then we can compute the fitness
  f = 0
  for i, item in enumerate(solution[0]):
    # all the items wich are not left out contribute to the fitness
    if not item:
      f += VALUES[i]    # add the value of the take item

  return f





In [ ]:
# generate a starting solution where each elelement is not taken
# there is an additional knapsack that represents the left out items
solution = np.array([
    [False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS + 1)], dtype=np.bool
)
# the row 0 of the solution represents the tleft out items

for i in range(NUM_ITEMS):
  solution[0][i] = True
  # to randomize it: solution[randint(0, NUM_KNAPSACKS-1)][i] = True


f = fitness(solution)
# for x in tqdm(range(10000)):
for x in range(10000):
  new_sol = tweak(solution)
  f_new = fitness(new_sol)
  if f_new > f:
    print(f"New fitness: {f_new}")
    f = f_new
    solution = new_sol
  elif f_new == f:
    solution = new_sol
  elif f_new == -2:
    raise Exception("Algorithm flaw: an item appears in multiple knapsacks")

print(f"Fitness: {f}, Solution:\n{solution}")



New fitness: 4375
New fitness: 16983
New fitness: 468108
New fitness: 613286
New fitness: 783671
New fitness: 790589
New fitness: 876702
New fitness: 890765
New fitness: 915194
New fitness: 917911
New fitness: 926252
New fitness: 960444
New fitness: 967380
New fitness: 988447
New fitness: 991524
New fitness: 1006884
New fitness: 1032969
New fitness: 1044662
New fitness: 1046066
New fitness: 1048984
New fitness: 1054114
New fitness: 1070313
New fitness: 1077493
New fitness: 1100137
New fitness: 1132883
New fitness: 1133211
New fitness: 1137514
New fitness: 1140061
New fitness: 1146965
New fitness: 1148184
New fitness: 1161617
New fitness: 1166535
New fitness: 1173946
New fitness: 1175861
New fitness: 1188859
New fitness: 1215274
New fitness: 1221165
New fitness: 1223781
New fitness: 1235244
New fitness: 1239280
New fitness: 1247409
New fitness: 1247747
New fitness: 1252943
New fitness: 1259343
New fitness: 1259726
New fitness: 1268126
New fitness: 1292326
New fitness: 1307439
New fitnes

# Three problems for testing

In [10]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [18]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)